<a href="https://colab.research.google.com/github/Filipriec/zaklady_pythonu/blob/main/10_datove_struktury2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dátové štruktúry 2
## Array (dynamické pole)

### Čo je dynamické pole?

Dynamické pole (v Pythone `list`) je štruktúra, ktorá umožňuje:
- **Rýchly prístup k prvkom podľa indexu** (O(1))
- **Flexibilnú veľkosť** (automaticky rastie pri pridávaní)
  
Napriek názvu *pole* je v Pythone implementované ako **dynamické pole s over-allocation** (pri plnení sa kapacita zdvojnásobí(približne)).

---


### Základné operácie a časová zložitosť

| Operácia           | Časová zložitosť | Poznámka                      |
|--------------------|------------------|-------------------------------|
| `a[i]`             | O(1)             | Indexovaný prístup            |
| `a.append(x)`      | O(1) amortizované | Pridanie na koniec            |
| `a.insert(i, x)`   | O(n)             | Pomalé – posúva prvky         |
| `a.pop()`          | O(1)             | Odstránenie z konca           |
| `a.pop(0)`         | O(n)             | Odstránenie zo začiatku       |
| `a.remove(x)`      | O(n)             | Hľadanie + posun prvkov       |
| `x in a`           | O(n)             | Lineárne vyhľadávanie         |

---

## Praktické využitie Python `list`

### 1. Efektívne operácie


In [5]:
# Správne: O(1) operácie
data = []
data.append(10)      # Rýchle
data.append(20)
last = data.pop()    # Rýchle
print(data)

[10]


In [14]:
# Pomalé: O(n) operácie
data = [1, 2, 3, 4, 5]
data.insert(0, 0)    # Pomalé pre veľké polia - musí posunúť všetko
print(data)
data.pop(0)          # Pomalé - musí posunúť všetko doľava
print(data)

[0, 1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]


### Prečo je `insert(0)` a `pop(0)` pomalé?

Pri vložení na začiatok sa musia **všetky prvky posunúť doprava**:

```
insert(0, "X"):

Pred:   ┌───┬───┬───┬───┐
        │ a │ b │ c │ d │
        └───┴───┴───┴───┘
          ↓   ↓   ↓   ↓     (posun všetkých prvkov)
Po:     ┌───┬───┬───┬───┬───┐
        │ X │ a │ b │ c │ d │
        └───┴───┴───┴───┴───┘
```

Čím viac prvkov, tým viac práce → O(n).

---



### 2. Rast kapacity
Python `list` alokuje viac miesta v pamäti, ako je potrebné, aby sa vyhol častým realokáciám:

In [15]:
import sys

lst = []
predchadzajuca = 0

for i in range(20):
    velkost = sys.getsizeof(lst)
    if velkost > predchadzajuca:
        print(f"Prvkov: {len(lst):2}, Veľkosť: {velkost} bajtov ← REALOKÁCIA")
    else:
        print(f"Prvkov: {len(lst):2}, Veľkosť: {velkost} bajtov")
    predchadzajuca = velkost
    lst.append(i)

Prvkov:  0, Veľkosť: 56 bajtov ← REALOKÁCIA
Prvkov:  1, Veľkosť: 88 bajtov ← REALOKÁCIA
Prvkov:  2, Veľkosť: 88 bajtov
Prvkov:  3, Veľkosť: 88 bajtov
Prvkov:  4, Veľkosť: 88 bajtov
Prvkov:  5, Veľkosť: 120 bajtov ← REALOKÁCIA
Prvkov:  6, Veľkosť: 120 bajtov
Prvkov:  7, Veľkosť: 120 bajtov
Prvkov:  8, Veľkosť: 120 bajtov
Prvkov:  9, Veľkosť: 184 bajtov ← REALOKÁCIA
Prvkov: 10, Veľkosť: 184 bajtov
Prvkov: 11, Veľkosť: 184 bajtov
Prvkov: 12, Veľkosť: 184 bajtov
Prvkov: 13, Veľkosť: 184 bajtov
Prvkov: 14, Veľkosť: 184 bajtov
Prvkov: 15, Veľkosť: 184 bajtov
Prvkov: 16, Veľkosť: 184 bajtov
Prvkov: 17, Veľkosť: 248 bajtov ← REALOKÁCIA
Prvkov: 18, Veľkosť: 248 bajtov
Prvkov: 19, Veľkosť: 248 bajtov


## Kedy nepoužiť `list`?

### Prípad 1: Časté operácie na začiatku

In [12]:
from collections import deque

# Zlé pre list, dobré pre deque
items = deque(maxlen=1000)  # Kruhový buffer
items.appendleft(10)        # O(1)
items.popleft()            # O(1)

10

## Kedy nepoužiť `list`?

### Časté operácie na začiatku listu → použite `deque`

In [18]:
from collections import deque

# deque má O(1) operácie na OBOCH koncoch
d = deque([1, 2, 3])

d.appendleft(0)   # O(1)
print(d)
d.popleft()       # O(1)
print(d)

deque([0, 1, 2, 3])
deque([1, 2, 3])



## Benchmark: `list` vs `deque`

In [19]:
from collections import deque
import time

def zmeraj(operacia, popis):
    start = time.perf_counter()
    operacia()
    koniec = time.perf_counter()
    print(f"{popis}: {(koniec - start)*1000:.2f} ms")

n = 50_000

print(f"Porovnanie {n:,} operácií na ZAČIATKU:\n")

def test_list():
    lst = []
    for i in range(n):
        lst.insert(0, i)

def test_deque():
    d = deque()
    for i in range(n):
        d.appendleft(i)

zmeraj(test_list, "list.insert(0)")
zmeraj(test_deque, "deque.appendleft()")

Porovnanie 50,000 operácií na ZAČIATKU:

list.insert(0): 781.92 ms
deque.appendleft(): 2.54 ms


---

## Porovnanie `list` vs `deque`

| Operácia | List | Deque | Optimálnejšie? |
|----------|------|-------|-------------|
| `a[i]` prístup | O(1) | O(n) | **List** |
| Pridať na koniec | O(1) | O(1) | Rovnaké |
| Pridať na začiatok | O(n) | O(1) | **Deque** |
| Odobrať z konca | O(1) | O(1) | Rovnaké |
| Odobrať zo začiatku | O(n) | O(1) | **Deque** |
| Hľadať `x in a` | O(n) | O(n) | Rovnaké |

---